# Solving problems by Searching

This notebook serves as supporting material for topics covered in **Chapter 3 - Solving Problems by Searching** and **Chapter 4 - Beyond Classical Search** from the book *Artificial Intelligence: A Modern Approach.* This notebook uses implementations from [search.py](https://github.com/aimacode/aima-python/blob/master/search.py) module. Let's start by importing everything from search module.

In [91]:

import random
import bisect
# Needed to hide warnings in the matplotlib sections
import warnings
#from numpy import *
warnings.filterwarnings("ignore")

For visualisations, we use networkx and matplotlib to show the map in the notebook and we use ipywidgets to interact with the map to see how the searching algorithm works. These are imported as required in `notebook.py`.

## CONTENTS

* Genetic Algorithm


## GENETIC ALGORITHM

Genetic algorithms (or GA) are inspired by natural evolution and are particularly useful in optimization and search problems with large state spaces.

Given a problem, algorithms in the domain make use of a *population* of solutions (also called *states*), where each solution/state represents a feasible solution. At each iteration (often called *generation*), the population gets updated using methods inspired by biology and evolution, like *crossover*, *mutation* and *natural selection*.

PERMUTATION ENCODING

GA Parameters:
We now need to define the maximum size of each population. Larger populations have more variation but are computationally more  expensive to run algorithms on.
As our population is not very large, we can afford to keep a relatively large mutation rate.
Termination after a predefined number of generations.
N is the size of the chromosmes, and [0,..,N-1] is the alphabet

To generate `ngen` number of generations, we run a `for` loop `ngen` number of times. After each generation, we calculate the fitness of the best individual of the generation and compare it to the value of `f_thres` using the `fitness_threshold` function. After every generation, we print out the best individual of the generation and the corresponding fitness value. Lets now write a function to do this.

In [92]:
def genetic_algorithm_stepwise(population, fitness_fn, gene_pool, ngen=1200, pmut=0.1):
    for generation in range(int(ngen)):
        # Elitism may be here - ADDED
        previous_best = max(population, key=fitness_fn)
        population = [mutate2(crossover(*select(2, population, fitness_fn)), pmut) for i in range(len(population)-1)]
        population.append(previous_best)
        # stores the individual genome with the highest fitness in the current population
        current_best = max(population, key=fitness_fn)
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tBest Fitness: {fitness_fn(current_best)}\r')
        print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tBest Fitness: {fitness_fn(current_best)}\t\tAverage Fitness: {sum(map(fitness_fn, population)) / len(population)}\r')
        #print(sum(map(fitness_fn, population)) / len(population), end=";") # imprime el valor medio del fitness de la población actual
        #print(fitness_fn(current_best)) # imprime el fitness del mejor individuo de la población actual
    return max(population, key=fitness_fn)


def genetic_algorithm_stepwise2(population, fitness_fn, gene_pool, ngen=1000, pmut=0.1, pLSA=0.1):
    for generation in range(int(ngen)):
        next_population = []

        # 'previous_best' guarda el mejor individuo de la generación anterior, es decir, agregamos elitismo al AG
        previous_best = max(population, key=fitness_fn)
        next_population.append(previous_best)
        
        # Generar 'len(population)-1' hijos para la población nueva ('-1' porque tenemos elitismo, es decir, reutiizamos 1 individuo de la pobación anterior)
        for i in range(len(population)-1):
            parent1, parent2 = select(2, population, fitness_fn)    # Selección -> Qué padres se seleccionan
            child = crossover(parent1, parent2)                     # Cruce     -> Cómo se reproducen
            child = mutate2(child, pmut)                            # Mutación  -> Aplicar una mutación al hijo con una probabilidad
            child = LSA(child, pLSA)
            next_population.append(child)

        population = next_population

        # stores the individual genome with the highest fitness in the current population
        current_best = max(population, key=fitness_fn)

        # DEBUG MESSAGES
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tBest Fitness: {fitness_fn(current_best)}\r')
        #print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tBest Fitness: {fitness_fn(current_best)}\t\tAverage Fitness: {sum(map(fitness_fn, population)) / len(population)}\r')
        print(str(sum(map(fitness_fn, population)) / len(population)).replace(".",",")) # imprime el valor medio del fitness de la población actual
        #print(str(fitness_fn(current_best)).replace(".", ",")) # imprime el fitness del mejor individuo de la población actual
    return max(population, key=fitness_fn)    

def LSA(child, pLSA):
    # Probabilidad de realizar la búsqueda local
    if random.uniform(0, 1) >= pLSA:
        return child

    prev_fitness = fitness_fn(child)
    actual_child = child.copy()
    iterations = 0
    max_iterations = 100  # Limite de iteraciones para evitar bucles infinitos

    while iterations < max_iterations:
        iterations += 1
        x = random.randint(0, len(actual_child) - 1)  # Cambiado a 0 para incluir el primer índice

        neighbors = []

        for i in range(N):
            next_index = (x + i) % len(actual_child)

            new_child = actual_child.copy()
            new_child[x], new_child[next_index] = new_child[next_index], new_child[x]
            neighbors.append(new_child)

        #print(neighbors)
       
        best_neighbor = max(neighbors, key=fitness_fn)

        if fitness_fn(best_neighbor) <= prev_fitness:  # Si no ha habido mejora
            return actual_child
        
        actual_child = best_neighbor

    return actual_child

def fitness_fn(sample):
    # initialize fitness to 0
    fitness = 0
    for i in range(len(sample) + 1):
        ciudad1 = 0 if i == 0 else sample[i - 1]
        ciudad2 = sample[i] if i < len(sample) else 0
        fitness += grafo.Dist[ciudad1][ciudad2]
    return 1/fitness # porque el mejor fitness es el menor coste

def init_population(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    for _ in range(pop_number):
        # Shuffle the gene pool and take the first pool_size elements as an individual
        v = gene_pool[:]
        random.shuffle(v)
        population.append(v)
    return population

# Ejercicio 5
def init_population_NN(pop_number, gene_pool):
    # a chromosome is a random permutation of the alphabet
    population = []
    
    # Para cada ciudad
    for i in range(1, pop_number):
        # Empezamos desde la ciudad i
        non_visited = gene_pool.copy()
        non_visited.remove(i)
        actual = i
        path = [actual]
        
        # Hasta que no queden ciudades por visitar
        while non_visited:
            min_city = -1
            min_cost = float('inf')            
            
            # Encontramos el camino con menor coste
            for n in non_visited:
                cost = grafo.Dist[actual][n]
                if cost < min_cost:
                    min_city = n
                    min_cost = cost
            # Añadimos la ciudad con menor coste y la borramos de los no visitados
            path.append(min_city)
            non_visited.remove(min_city)
            actual = min_city  # Actualizamos la ciudad actual
            
        population.append(path)

    return population

def select(r, population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    #scaling here
    ##############################################
    scaled = list(fitnesses)
    min_v = min(scaled) - 0.00001
    scaled = map(lambda x: x - min_v, scaled)
    ##############################################
    sampler = weighted_sampler(population, scaled)
    return [sampler() for i in range(r)]

def weighted_sampler(seq, weights):
    """Return a random-sample function that picks from seq weighted by weights."""
    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]
    # bisect(a,x) -> insertion position of a in a sorted list x - AL REVES

# Ejercicio 4
# Se combinan los padres, alternativamente uno y otro
def alter_crossover(x, y):
    child = []
    for i in range(1, N - 1):
        child[i - 1] = x[i - 1]
        child[i] = x[i]
    return child

def extreme_crossover(x,y):
    child = [-1] * N
    used = set()

    usingX, usingY = True, True
    
    for i in range(N):
        if usingX and x[i] not in used:
            child[i] = x[i]
            used.add(x[i])
        else:
            if not usingY:
                break
            usingX = False

        if usingY and y[N-1-i] not in used:
            child[N-1-i] = y[N-1-i]
            used.add(y[N-1-i])
        else:
            if not usingX:
                break
            usingY = False

    not_used = [i for i in y if i not in used]

    for i in range(N):
        if child[i] == -1:
            child[i] = not_used.pop()

    return child

def random_crossover(x, y):
    child = x.copy()
    random.shuffle(child)
    return child

def uniform_crossover(x, y):
    # x, y permutations of the alphabet
    n = 0
    child = [-1] * N
    indexes = [0] * N
    # de x se copian los valores de las posiciones con indexex[i] == 1 en las mismas posiciones en child
    for i in range(N):
        indexes[i] = random.randint(0,1) 
        if indexes[i] == 1:
            child[i] = x[i]
            n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0 # indice en y
    k = 0 # indice en child
    for t in range(N-n):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1   
    return child

def order_crossover(x, y): # OX
    n = 0
    child = [-1] * N
    p1 = random.randint(0, N-1)
    p2 = random.randint(0, N-1)
    # de x se copian los valores situados en el rango de posiciones [p1, p2]
    # si p1 > p2, se copian todos los valores de x excepto los de las posiciones [p2, p1]
    if p2 >= p1:
        for i in range(p1, p2+1):
            child[i] = x[i]
            n += 1
    else: # p1 > p2
        for i in range(N):
            if i < p2 or i > p1:
                child[i] = x[i]
                n += 1
    # El resto (N-n) se copia de y en su orden relativo, desde el principio
    i = 0 # indice en y
    k = 0 # indice en child
    for t in range(N-n):
        while y[i] in child[:]:
            i += 1
        while child[k] != -1:
            k += 1
        child[k] = y[i]
        i += 1   
    return child

def crossover(x,y):
    return extreme_crossover(x,y)

def mutate2(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    i, j = random.sample(range(N), 2)
    x[i], x[j] = x[j], x[i]
    return x

Para adaptar el algoritmo anterior para resolver el TSP, solamente tenemos que definir la función fitness_fn: para cada permutación devuelve la inversa del coste del tour que representa la permutación. Para evitar soluciones repetidas, podemos prescindir del 0 en las permutaciones y suponer que la primera y ultima de las ciudades de la permutación son las que están conectadas con la ciudad incial


In [93]:
# Datos de instancias en formato EDGE_WEIGHT_TYPE de la TSPLIB
prueba = [ 
    0,
    1, 0,
    6, 2, 0,
    4, 5, 3, 0,
]

ejemploClase = [ 
    0,
    21, 0,
    12, 7, 0,
    15, 32, 5, 0,
    113, 25, 18, 180, 0,
    92, 9, 20, 39, 17, 0
]

gr21 = [
    0, 
    510, 0, 
    635, 355, 0, 
    91, 415, 605, 0, 
    385, 585, 390, 350, 0, 
    155, 475, 495, 120, 240, 0, 
    110, 480, 570, 78, 320, 96, 0, 
    130, 500, 540, 97, 285, 36, 29, 0, 
    490, 605, 295, 460, 120, 350, 425, 390, 0, 
    370, 320, 700, 280, 590, 365, 350, 370, 625, 0, 
    155, 380, 640, 63, 430, 200, 160, 175, 535, 240, 0, 
    68, 440, 575, 27, 320, 91, 48, 67, 430, 300, 90, 0, 
    610, 360, 705, 520, 835, 605, 590, 610, 865, 250, 480, 545, 0, 
    655, 235, 585, 555, 750, 615, 625, 645, 775, 285, 515, 585, 190, 0, 
    480, 81, 435, 380, 575, 440, 455, 465, 600, 245, 345, 415, 295, 170, 0, 
    265, 480, 420, 235, 125, 125, 200, 165, 230, 475, 310, 205, 715, 650, 475, 0, 
    255, 440, 755, 235, 650, 370, 320, 350, 680, 150, 175, 265, 400, 435, 385, 485, 0, 
    450, 270, 625, 345, 660, 430, 420, 440, 690, 77, 310, 380, 180, 215, 190, 545, 225, 0, 
    170, 445, 750, 160, 495, 265, 220, 240, 600, 235, 125, 170, 485, 525, 405, 375, 87, 315, 0, 
    240, 290, 590, 140, 480, 255, 205, 220, 515, 150, 100, 170, 390, 425, 255, 395, 205, 220, 155, 0, 
    380, 140, 495, 280, 480, 340, 350, 370, 505, 185, 240, 310, 345, 280, 105, 380, 280, 165, 305, 150, 0,
]

gr17 = [
    0, 
    633, 0, 
    257, 390, 0, 
    91, 661, 228, 0, 
    412, 227, 169, 383, 0, 
    150, 488, 112, 120, 267, 0, 
    80, 572, 196, 77, 351, 63, 0, 
    134, 530, 154, 105, 309, 34, 29, 0, 
    259, 555, 372, 175, 338, 264, 232, 249, 0, 
    505, 289, 262, 476, 196, 360, 444, 402, 495, 0, 
    353, 282, 110, 324, 61, 208, 292, 250, 352, 154, 0, 
    324, 638, 437, 240, 421, 329, 297, 314, 95, 578, 435, 0, 
    70, 567, 191, 27, 346, 83, 47, 68, 189, 439, 287, 254, 0, 
    211, 466, 74, 182, 243, 105, 150, 108, 326, 336, 184, 391, 145, 0, 
    268, 420, 53, 239, 199, 123, 207, 165, 383, 240, 140, 448, 202, 57, 0, 
    246, 745, 472, 237, 528, 364, 332, 349, 202, 685, 542, 157, 289, 426, 483, 0, 
    121, 518, 142, 84, 297, 35, 29, 36, 236, 390, 238, 301, 55, 96, 153, 336, 0,
 ]

class Grafo:
    """Un Grafo es un array bidimensional de tamaño N*N, siendo N el número de ciudades,
    representadas por los valores 0,...,N-1; 0 se toma como ciudad de partida.
    El valor de la posición (i,j) es la distancia entre las ciudades i y j,
     que es el mismo que la distancia entre j e i """
    def __init__(self,lista):
        "N es el número de ciudades correspondiente a la lista de valores de una instancia en la sintaxis EDGE_WEIGHT_TYPE"
        self.N = int(((8*len(lista)+1)**0.5)-1)/2
        self.ciudades = list(range(int(self.N)))
        self.Dist = [[0]*int(self.N) for i in self.ciudades]
        x_acc = 0
        for x in range(int(self.N)):
            x_acc += x
            for y in range(x+1):
                self.Dist[x][y] = self.Dist[y][x] = lista[x_acc+y]

def fitness_fn(sample):
    # initialize fitness to 0
    fitness = 0
    for i in range(len(sample) + 1):
        ciudad1 = 0 if i == 0 else sample[i - 1]
        ciudad2 = sample[i] if i < len(sample) else 0
        fitness += grafo.Dist[ciudad1][ciudad2]
    return 1/fitness # porque el mejor fitness es el menor coste
         

The function defined above is essentially the same as the one defined in `search.py` with the added functionality of printing out the data of each generation.

En el bloque siguiente se introduce la instancia que queremos resolver y los parámetos del algoritmo genético

In [94]:
# Problem instance
grafo = Grafo(gr17) # problem instance
# Parameters . . . 
max_population = int(grafo.N)
def crossover(x,y):
    return order_crossover(x,y)
mutation_rate = 0.07 # 7% of the chromosones are mutated
ngen = 1000 # number of generations
N = int(grafo.N) # chormosome size
gene_pool = list(range(N)) # alphabet
gene_pool.remove(0)
N -= 1 # 0 es la ciudad inicial, no esta en el cromosoma
print(gene_pool)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


We have defined all the required functions and variables. Let's now create a new population and test the function we wrote above.

In [95]:
population = init_population_NN(max_population, gene_pool)
#print(population)
solution = genetic_algorithm_stepwise2(population, fitness_fn, gene_pool, ngen, mutation_rate)
print("Solution: ")
print(solution)
print("Fitness: ")
fitness = fitness_fn(solution)
print(fitness)
print("Cost: ")
print(1/fitness)

0,00038865603828751435
0,00037895187067247
0,0003944638693920994
0,00039996896496209683
0,0004248481077999832
0,0004389133486002451
0,00044238111329428215
0,000441352357771111
0,00044884690965005166
0,00046358854992505256
0,00046253182416520957
0,00046802951122383726
0,00047135884809025564
0,00047486247838977603
0,00047483933442040013
0,00047625095939545333
0,00047871005521164464
0,0004690006126197361
0,00046967877355818486
0,0004763665970695887
0,0004778824929211624
0,00046612881883671193
0,00046545561391827145
0,0004739380972865324
0,00046925736247160465
0,0004704277612201893
0,00047853534093668203
0,00047872358232539143
0,0004718901987701402
0,00047961630695443646
0,0004731188752979585
0,0004595312143524206
0,00047961630695443646
0,00047355000467873596
0,00047179352249552414
0,00047853534093668203
0,00046473272650042035
0,00046115460244857044
0,00047384632959782445
0,00047611020606780314
0,0004699414982806266
0,0004792874534693904
0,0004715366053141638
0,00047789526064501023
0,00046